# openp

## README

this notebook implements open-pseudonymiser in jupyter

_note: the kernel needs to be manually restarted before you can re-run this notebook - if you see_ `VM is already running...` _then you need to restart the kernel_

## dependencies

### java

as installed by [scoop](https://scoop.sh)

```
scoop bucket add java
scoop install java/zulu-jdk
```

## ant

if you want to build the jar (hint, you should be able to base your tests on the current codebase, so it's worth building the libray jar)

```
scoop install main/ant
```

## pyjnius

```sh
pip install pyjnius
```

# code

python imports

In [1]:
import os
from pathlib import Path
# pip install pyjnius
import jnius_config


setting up java library: `OpenPseudonymiserCryptoLib.jar`

In [2]:
def set_java_lib(jar_path):
  """set location of OpenPseudonymiserCryptoLib.jar 
  note - you will need to have built (or downloaded) it first

  Args:
      jar_path (string): path to lib jar

  Returns:
      string: location
  """  
  jnius_config.set_classpath(".", jar_path)
  return jar_path

import the `crypto` lib from the jar

In [3]:
def import_java_crypto():
  """import the crypto lib from java
  """
  from jnius import autoclass
  Crypto = autoclass("OpenPseudonymiser.Crypto")
  crypto = Crypto()
  return crypto

give the crypto lib an ecryption salt that has itself been encrypted

In [4]:
def set_encrypted_salt(salt_path, crypto):
  """ usually ./mackerel.EncryptedSalt"

  Args:
      salt_path (string): path to encrypted salt
      crypto (object): the java crypto class

  Returns:
      object: the java crypto class
  """
  from jnius import autoclass
  File = autoclass("java.io.File")
  file = File(salt_path)
  crypto.SetEncryptedSalt(file)
  return crypto

### test

this test creates a dataset, and tests that encrypting a known dataset with a salt encrypted by someone else (trusted keyholder) produces the same encrypted output

In [5]:
def test_crypto(crypto):
  from jnius import autoclass
  Map = autoclass("java.util.Map")
  TreeMap = autoclass("java.util.TreeMap")
  treeMap = TreeMap()
  treeMap.put("DOB", "29.11.1973")
  treeMap.put("NHSNumber", "943 476 5919")

  # test encryption matches expected outcome 
  try:
      assert "ED72F814B7905F3D3958749FA90FE657C101EC657402783DB68CBE3513E76087" == crypto.GetDigest(treeMap)
      print("test passed", crypto.GetDigest(treeMap))
  except(Exception):
      print(Exception)
      print("test failed")


### put it all together

In [6]:
# set crypto lib
jar_path = os.path.join(
    Path(os.getcwd()).absolute()
    , "dist"
    , "*"
)
# jar_path = os.path.join(Path(os.getcwd()).absolute(), "dist", "*")
set_java_lib(jar_path)

# get crypto class
crypto = import_java_crypto()  

# set salt
salt_path = "./mackerel.EncryptedSalt"
set_encrypted_salt(salt_path, crypto)

# some test data
test_crypto(crypto)


test passed ED72F814B7905F3D3958749FA90FE657C101EC657402783DB68CBE3513E76087
